In [ ]:
# default_exp tabular.preprocessing

In [ ]:
#hide
from nbdev.showdoc import *

# Preprocessor for tabular models

In [ ]:
#export
from fastai.tabular.all import *
from fastai.data.all import *
from fastai.vision.data import get_grid
from enveco.data.las import *
from enveco.data.image import *
import matplotlib.patches as mpl_patches
from typing import Tuple
from fastai.metrics import *

## Preprocessing

Preprocess csv-files of our field plot data into `TabularPandas` to feed into models.

In [ ]:
#export

def get_lidar_feature(row, path, feature_func, min_h:float=1.5, mask_plot=True):
    "Function for LiDAR opening and processing steps"
    las_data = las_to_df(f'{path}/{row.sampleplotid}.las')
    if mask_plot == True: las_data = mask_plot_from_lidar(las_data, row.x, row.y)
    features = feature_func(las_data, min_h)
    return features

In [ ]:
#export

def get_image_procs(row, path, radius=31, mask_plot=True):
    "Function for tif opening and processing steps"
    image_data = open_geotiff(f'{path}/{row.sampleplotid}.tif')
    if mask_plot == True: image_data = mask_plot_from_image(image_data, radius=radius)
    metrics = calc_image_metrics(image_data)
    return metrics

In [ ]:
# export

class EnvecoPreprocessor():
    
    def __init__(self, train_path, valid_path, test_path, **kwargs):
        self.train_df = pd.read_csv(train_path)
        self.train_df = self.train_df.rename(columns = lambda x: re.sub('[\.]+', '_', x))
        self.valid_df = pd.read_csv(valid_path)
        self.valid_df = self.valid_df.rename(columns = lambda x: re.sub('[\.]+', '_', x))
        self.test_df = pd.read_csv(test_path)
        self.test_df = self.test_df.rename(columns = lambda x: re.sub('[\.]+', '_', x))
        self.train_df['is_valid'] = 0
        self.valid_df['is_valid'] = 1
        self.train_val_df = pd.concat((self.train_df, self.valid_df))
        
        
    def preprocess_lidar(self, target_col, path, min_h:float=1.5, mask_plot:bool=True, height_features:bool=True,
                         point_features:bool=True, intensity_features:bool=True, height_quantiles:bool=True,
                         point_proportions:bool=True, canopy_densities:bool=True, normalize:bool=True,
                         log_y:bool=False) -> Tuple[TabularPandas, TabularPandas]:
        "Preprocess data and return (train_val, test) -tuple. Optionally log-transform target column with np.log1p"
        trainval = self.train_val_df.copy()
        test = self.test_df.copy()
        feature_cols = []
        if height_features:
            print('Adding height based features')
            trainval[height_cols] = trainval.apply(lambda row: get_lidar_feature(row, path, calc_height_features, 
                                                                                 min_h, mask_plot), 
                                                   axis=1, result_type='expand')
            test[height_cols] = test.apply(lambda row: get_lidar_feature(row, path, calc_height_features, 
                                                                         min_h, mask_plot), 
                                           axis=1, result_type='expand')
            feature_cols.extend(height_cols)

        if point_features:
            print('Adding point distribution based features')
            trainval[point_cols] = trainval.apply(lambda row: get_lidar_feature(row, path, calc_point_features, 
                                                                                min_h, mask_plot), 
                                                   axis=1, result_type='expand')
            test[point_cols] = test.apply(lambda row: get_lidar_feature(row, path, calc_point_features, 
                                                                                 min_h, mask_plot), 
                                           axis=1, result_type='expand')
            feature_cols.extend(point_cols)
            
        if intensity_features:
            print('Adding intensity based features')
            trainval[intensity_cols] = trainval.apply(lambda row: get_lidar_feature(row, path, calc_intensity_features, 
                                                                                    min_h, mask_plot), 
                                                   axis=1, result_type='expand')
            test[intensity_cols] = test.apply(lambda row: get_lidar_feature(row, path, calc_intensity_features, 
                                                                            min_h, mask_plot), 
                                              axis=1, result_type='expand')
            feature_cols.extend(intensity_cols)
            
        if height_quantiles:
            print('Adding height quantiles')
            trainval[quantile_cols] = trainval.apply(lambda row: get_lidar_feature(row, path, calc_height_quantiles, 
                                                                                  min_h, mask_plot), 
                                                   axis=1, result_type='expand')
            test[quantile_cols] = test.apply(lambda row: get_lidar_feature(row, path, calc_height_quantiles,
                                                                           min_h, mask_plot), 
                                             axis=1, result_type='expand')
            feature_cols.extend(quantile_cols)
            
        if point_proportions:
            print('Adding point proportions')
            trainval[proportion_cols] = trainval.apply(lambda row: get_lidar_feature(row, path, calc_point_proportions, 
                                                                                    min_h, mask_plot), 
                                                       axis=1, result_type='expand')
            test[proportion_cols] = test.apply(lambda row: get_lidar_feature(row, path, calc_point_proportions, 
                                                                                 min_h, mask_plot), 
                                               axis=1, result_type='expand')
            feature_cols.extend(proportion_cols)
            
        if canopy_densities:
            print('Adding canopy densities')
            trainval[density_cols] = trainval.apply(lambda row: get_lidar_feature(row, path, calc_canopy_densities, 
                                                                                    min_h, mask_plot), 
                                                       axis=1, result_type='expand')
            test[density_cols] = test.apply(lambda row: get_lidar_feature(row, path, calc_canopy_densities, 
                                                                                 min_h, mask_plot), 
                                               axis=1, result_type='expand')
            feature_cols.extend(density_cols)   
        
        if log_y:
            trainval[target_col] = np.log1p(trainval[target_col])
            test[target_col] = np.log1p(test[target_col])
        
        means = []
        stds = []
        #for c in feature_cols:
        #    means.append(trainval[trainval.is_valid==0][c].mean())
        #    stds.append(trainval[trainval.is_valid==0][c].std())
        #norm_stats = np.array((means,stds))
        procs = None
        if normalize:
            procs = [Normalize]#.from_stats(*norm_stats)]
        trainval_tb = TabularPandas(trainval, procs=procs,
                                    cont_names=feature_cols, y_names=target_col,
                                    splits=ColSplitter(col='is_valid')(trainval))
        test_tb = TabularPandas(test, procs=procs,
                                cont_names=feature_cols, y_names=target_col)
        return trainval_tb, test_tb
    
    def preprocess_image(self, target_col, path, radius:int=31, mask_plot=True) -> Tuple[TabularPandas, TabularPandas]:
        "Preprocess dataframes and return (train_val, test) -tuple"
        # TODO
        pass
    
    def preprocess_lidar_and_image(self, target_col, path, min_h:float=1.5, radius:int=31, 
                                   mask_plot:bool=True) -> Tuple[TabularPandas, TabularPandas]:
        "Preprocess dataframes and return (train_val, test) -tuple"
        # TODO
        pass

## Ensemble

Ensemble of tabular learners with `n_models`. 

In [ ]:
# export
class ANNEnsemble():
    
    def __init__(self, dls, metrics, y_range:tuple=None, n_models=10, **learner_kwargs):
        "Create an ensemble of ANN models. TODO add option to use different parameters or different split for dataset"
        self.dls = dls
        self.metrics = metrics
        self.models = []
        for _ in range(n_models):
            # This way instead of list comprehension because possible model-specific settings            
            self.models.append(tabular_learner(dls, metrics=metrics, y_range=y_range, **learner_kwargs))
        
    def fit_one_cycle(self, n_iterations, max_lr):
        "Fit the models"
        for m in self.models:
            m.fit_one_cycle(n_iterations, max_lr=max_lr)
            
    def validate(self, dl=None) -> pd.DataFrame:
        "Validate all models individually and as an ensemble"
        if dl is None: dl=self.dls[1]
        model_results = torch.cat([m.get_preds(reorder=False, dl=dl)[0] for m in self.models], dim=-1)
        ensemble_results = model_results.sum(axis=-1) / len(self.models)
        res_df = pd.DataFrame(columns=['model_identifier'] + [m.name if hasattr(m, 'name') else m.__name__ for m in self.metrics])
        res_df.loc[0] = (['ensemble'] 
                         + [metric(ensemble_results, Tensor(dl.y.values)).item() for metric in self.metrics])
        for i in range(len(self.models)):
            res_df.loc[i+1] = ([i] 
                               + [metric(model_results[:,i], Tensor(dl.y.values)).item()  for metric in self.metrics])
        return res_df
    
    def get_ensemble_preds(self, ds_idx=1, dl=None, with_input=True, with_decoded=False, with_loss=False, act=None,
                           inner=False, reorder=False, cbs=None, **kwargs):
        "get_preds but ensemble results"
        if dl is None: dl=self.dls[1].new(shuffled=False, drop_last=False)
        if reorder and hasattr(dl, 'get_idxs'):
            idxs = dl.get_idxs()
            dl = dl.new(get_idxs = _ConstantFunc(idxs))
        model_results = []
        for m in self.models:
            model_results.append(m.get_preds(dl=dl, with_input=with_input, with_decoded=with_decoded, with_loss=with_loss,
                                             act=act, inner=inner, reorder=reorder, cbs=cbs, **kwargs))
        
        ensemble_results = []
        # iterate through results:
        obs_idx = 0
        if with_input: 
            ensemble_results.append(model_results[0][obs_idx])
            obs_idx += 1
        ensemble_results.append(sum([res[obs_idx] for res in model_results])/len(self.models))
        obs_idx += 1
        ensemble_results.append(model_results[0][obs_idx])
        obs_idx += 1
        if with_decoded:
            ensemble_results.append(model_results[0][obs_idx])
            obs_idx += 1
        if with_loss:
            ensemble_results.append(sum([res[obs_idx] for res in model_results])/len(self.models))
        return tuple(ensemble_results)
    
    def predict(self, item):
        model_results = [m.predict(item) for m in self.models]
        ensemble_results = sum([res[-1] for res in model_results])/len(self.models)
        ensemble_dec_results = sum([res[-2] for res in model_results])/len(self.models)
        return (model_results[0][0], ensemble_dec_results, ensemble_results)
    
    
    def export(self, path, pickle_protocol=2):
        "Save each Learner in the ensemble"
        pass

def load_ensemble(path, cpu=True) -> ANNEnsemble:
    # Read config and dls
    pass

In [ ]:
# hide

from nbdev.export import notebook2script
notebook2script()
!nbdev_build_docs

Converted 00_data.image.ipynb.
Converted 01_data.las.ipynb.
Converted 02_tabular.preprocessing.ipynb.
Converted 03_models.inception3dv3.ipynb.
Converted 04_interpretation.ipynb.
Converted 05_metrics.ipynb.
Converted 06_models.ensemble.ipynb.
Converted index.ipynb.
converting: /mnt/d/Users/E1005164/enveco/nbs/00_data.image.ipynb
[Errno 22] Invalid argument
converting: /mnt/d/Users/E1005164/enveco/nbs/01_data.las.ipynb
[Errno 22] Invalid argument
converting: /mnt/d/Users/E1005164/enveco/nbs/02_tabular.preprocessing.ipynb
[Errno 22] Invalid argument
Conversion failed on the following:              
00_data.image.ipynb
01_data.las.ipynb
02_tabular.preprocessing.ipynb
converting /mnt/d/Users/E1005164/enveco/nbs/index.ipynb to README.md
